The Climate at a Glance's API takes input on location, parameter, timescale, month, beginning year, end year, and format:

https://www.ncei.noaa.gov/access/monitoring/climate-at-a-glance/{scope}/time-series/{location}/{parameter}/{timescale}/{month}/{begYear}-{endYear}/data.{format}

Using the above url as `BASE_URL` this code inputs:
* location -> `stat_id` NOAA state codes 
* parameter -> `PARAM` tmax for maximum temperature, tmin for minimum temperature
* begYear -> `START_YEAR` 1925
* endYear -> `END_YEAR` 2025
* timescale -> 1, to take full year 
* month -> 0, to take all months for the year
* format -> JSON

In [2]:
## import packages

import requests
import pandas as pd
import io, json, time

In [17]:
## dictionary of NOAA state codes 1–51 (except 49 = Pacific Islands) based on Climate at a Glance page to fit API parameters
STATE_CODES = {
    1:"Alabama", 2:"Arizona", 3:"Arkansas", 4:"California", 5:"Colorado",
    6:"Connecticut", 7:"Delaware", 8:"Florida", 9:"Georgia", 10:"Idaho",
    11:"Illinois", 12:"Indiana", 13:"Iowa", 14:"Kansas", 15:"Kentucky",
    16:"Louisiana", 17:"Maine", 18:"Maryland", 19:"Massachusetts", 20:"Michigan",
    21:"Minnesota", 22:"Mississippi", 23:"Missouri", 24:"Montana", 25:"Nebraska",
    26:"Nevada", 27:"New Hampshire", 28:"New Jersey", 29:"New Mexico", 30:"New York",
    31:"North Carolina", 32:"North Dakota", 33:"Ohio", 34:"Oklahoma", 35:"Oregon",
    36:"Pennsylvania", 37:"Rhode Island", 38:"South Carolina", 39:"South Dakota",
    40:"Tennessee", 41:"Texas", 42:"Utah", 43:"Vermont", 44:"Virginia",
    45:"Washington", 46:"West Virginia", 47:"Wisconsin", 48:"Wyoming",
    50:"Alaska", 51:"Hawaii"
}

## define minimum and maximum temperature as search parameters, going back 100 years from 2025
PARAMS = ["tmax", "tmin"]
START_YEAR = 1925
END_YEAR = 2025   
BASE_URL = "https://www.ncei.noaa.gov/access/monitoring/climate-at-a-glance/statewide/time-series"

In [19]:
## use API to draw state-level data that will be used in the loop function

## for loop to pull temperature date for every state 

data = {}

for state_id, state_name in STATE_CODES.items():
    print(f"Getting {state_name}...")
    state_entry = {}
    
    for param in PARAMS:
        url = f"{BASE_URL}/{state_id}/{param}/1/0/{START_YEAR}-{END_YEAR}.json"
        try:
            r = requests.get(url, timeout=60)
            if r.status_code != 200:
                print(f"Failed to get {param} for state {state_id}: {r.status_code}")
                continue

            js = r.json()
            records = []

            ## organize the data and split out month from year 
            
            for date_str, value in js.get("data", {}).items():
                if isinstance(value, dict) and "value" in value: ## take value out as it is a nested key
                    value = value["value"]
                year = int(date_str[:4]) ## split the year out 
                month = int(date_str[4:6]) ## split the month out
                
                records.append({
                    "year": year, 
                    "month": month, 
                    "value": value})
            
            state_entry[param] = records

        ## error handling
        
        except Exception as e:
                    print(f"Error fetching {param} for {state_name}: {e}")
                    state_entry[param] = []

        ## space out the requests
        time.sleep(10)
    
    data[state_name] = state_entry

## save as JSON 

with open("state_temps.json", "w") as f:
    json.dump(data, f, indent=2)

print("Completed and saved state_temps.json")

Getting Alabama...
Getting Arizona...
Getting Arkansas...
Getting California...
Getting Colorado...
Getting Connecticut...
Getting Delaware...
Getting Florida...
Getting Georgia...
Getting Idaho...
Getting Illinois...
Getting Indiana...
Getting Iowa...
Getting Kansas...
Getting Kentucky...
Getting Louisiana...
Getting Maine...
Getting Maryland...
Getting Massachusetts...
Getting Michigan...
Getting Minnesota...
Getting Mississippi...
Getting Missouri...
Getting Montana...
Getting Nebraska...
Getting Nevada...
Getting New Hampshire...
Getting New Jersey...
Getting New Mexico...
Getting New York...
Getting North Carolina...
Getting North Dakota...
Getting Ohio...
Getting Oklahoma...
Getting Oregon...
Getting Pennsylvania...
Getting Rhode Island...
Getting South Carolina...
Getting South Dakota...
Getting Tennessee...
Getting Texas...
Getting Utah...
Getting Vermont...
Getting Virginia...
Getting Washington...
Getting West Virginia...
Getting Wisconsin...
Getting Wyoming...
Getting Alaska

In [21]:
## check that it works 

state = "Alabama"
param = "tmax"
data[state][param][:10] 

[{'year': 1925, 'month': 1, 'value': 57.2},
 {'year': 1925, 'month': 2, 'value': 64.3},
 {'year': 1925, 'month': 3, 'value': 70.8},
 {'year': 1925, 'month': 4, 'value': 80.8},
 {'year': 1925, 'month': 5, 'value': 82.6},
 {'year': 1925, 'month': 6, 'value': 93.3},
 {'year': 1925, 'month': 7, 'value': 93.6},
 {'year': 1925, 'month': 8, 'value': 93.3},
 {'year': 1925, 'month': 9, 'value': 96.8},
 {'year': 1925, 'month': 10, 'value': 75.8}]